In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Load data tables
Refer = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')
Discharge = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Historical/HistoricalDischarge.csv')
Precip = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Historical/HistoricalPrecip.csv')

# Standardize Date and River Names to be joined
Discharge['Date'] = pd.to_datetime(Discharge.DateTime).dt.date
Discharge = Discharge.set_index('Name').join(Refer.set_index('USGS Name')).reset_index()[[
    'Date','Value','VariableDescription','Name','RiverName','FishType']]

Precip = Precip.set_index('Name').join(Refer.set_index('Name')).reset_index()[[
    'Date','Precip','Name','RiverName','FishType']]

# Create Avg Daily Discharge
Avg_Discharge = Discharge.groupby(['Date','Name','RiverName','FishType']).mean().reset_index()

# Remove incorrect data
Avg_Discharge = Avg_Discharge.loc[Avg_Discharge.Value>0]

# Reduce data to Eel River only
eel = Avg_Discharge.loc[Avg_Discharge['Name']=='Eel River Scotia']
eel.reset_index(inplace=True)
eel = eel.drop(columns='index')

# Create column for slope of discharge
eel['CFS_Slope'] = eel.Value.diff()

# Retrieve values of Slope
SlopeValues = eel.CFS_Slope.values

# Define the threshold for what is a good day for fishing
#       - slope between day CFS values must be at least 1 STD from the mean 
border = eel.CFS_Slope.mean()+(eel.CFS_Slope.std()/2)

# Define troughs that are below the threshold
peaks, _ = find_peaks(-SlopeValues, height=border)

# Logic to build the best day flag
#       - if the value is a trough point or has a negative low and is above 1 STD from the mean then also consider a good day 
bestdays = []
for i in range(len(eel.Value)):
    if i in peaks:
        x = 1
    elif (eel.CFS_Slope[i]<0) & (eel.Value[i]>=border):
        x = 1
    else: x = 0
    bestdays.append(x)
eel['BestDays'] = bestdays

eel.head()

,Date,Name,RiverName,FishType,Value,CFS_Slope,BestDays
0,2020-10-19,Eel River Scotia,Eel,Steelhead,65.256250,NaN,0
1,2020-10-20,Eel River Scotia,Eel,Steelhead,65.536458,0.280208,0
2,2020-10-21,Eel River Scotia,Eel,Steelhead,64.127083,-1.409375,0
3,2020-10-22,Eel River Scotia,Eel,Steelhead,64.877083,0.750000,0
4,2020-10-23,Eel River Scotia,Eel,Steelhead,73.515625,8.638542,0


In [3]:
dates = eel.loc[eel.BestDays==1].Date.values
BestDayValues = eel.Value.loc[eel.BestDays==1].values

fig = px.line(eel, x="Date", y="Value", title='Eel River (Scotia) - Daily CFS')
fig.add_scatter(x=dates,y=BestDayValues,mode='markers',name='Good fishing days')
fig.show()

In [4]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

def FindBestDay(df):
    df['CFS_Slope'] = df.Value.diff()

    # Retrieve values of Slope
    SlopeValues = df.CFS_Slope.values

    # Define the threshold for what is a good day for fishing
    #       - slope between day CFS values must be at least 1 STD from the mean 
    border = df.CFS_Slope.mean()+(df.CFS_Slope.std()/2)

    # Define troughs that are below the threshold
    peaks, _ = find_peaks(-SlopeValues, height=border)

    # Logic to build the best day flag
    #       - if the value is a trough point or has a negative low and is above 1 STD from the mean then also consider a good day 
    bestdays = []
    for i in range(len(df.Value)):
        if i in peaks:
            x = 1
        elif (df.CFS_Slope[i]<0) & (df.Value[i]>=border):
            x = 1
        else: x = 0
        bestdays.append(x)
    df['BestDays'] = bestdays
    return df




,Date,Name,RiverName,FishType,Value,CFS_Slope,BestDays
0,2020-10-19,Eel River Scotia,Eel,Steelhead,65.256250,NaN,0
1,2020-10-20,Eel River Scotia,Eel,Steelhead,65.536458,0.280208,0
2,2020-10-21,Eel River Scotia,Eel,Steelhead,64.127083,-1.409375,0
3,2020-10-22,Eel River Scotia,Eel,Steelhead,64.877083,0.750000,0
4,2020-10-23,Eel River Scotia,Eel,Steelhead,73.515625,8.638542,0
